In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('states.csv')#, parse_dates=['ORDERDATE'], low_memory=False)
df

,COUNTY,STATE,High school graduate,Associates degree,Bachelors degree,High school graduate or higher,Bachelors degree or higher,"$25,000 to $34,999","$35,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999"
0,Alameda County,California,17.0,6.4,28.1,88.8,48.7,5.2,0.3,12.0,11.2,17.9
1,Alpine County,California,28.7,5.6,19.8,95.3,38.4,12.1,3.4,18.6,10.3,28.5
2,Modoc County,California,31.8,9.2,12.2,84.7,18.2,17.3,2.5,23.6,13.5,10.4
3,Amador County,California,30.0,13.4,12.3,91.2,18.1,9.9,2.1,19.3,13.8,16.2
4,Butte County,California,22.6,10.5,18.4,89.7,28.3,10.3,0.8,16.1,11.7,13.5
...,...,...,...,...,...,...,...,...,...,...,...,...
308,Wood County,Texas,33.1,7.2,12.2,85.6,19.2,12.0,1.6,17.5,12.4,12.1
309,Young County,Texas,30.4,4.8,16.5,84.8,23.6,12.3,2.9,17.9,10.8,15.0
310,Zapata County,Texas,30.6,5.1,11.1,64.7,12.2,16.3,4.2,11.4,9.7,4.9
311,Zavala County,Texas,35.0,6.5,7.0,75.7,12.4,12.4,5.7,14.2,8.9,3.7


In [2]:
#X = df.drop(columns = df.iloc[:,-5:]).copy()
#y = df.iloc[:,-5:]

In [3]:
# Let's say we want to split the data in 80:10:10 for train:valid:test dataset
train_size=0.8

X = df.drop(columns = ['$100,000 to $149,999']).copy()
y = df['$100,000 to $149,999']

# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train.columns if  
                        X_train[cname].dtype == "object"] # "object" | "category" for xgboost
 
# Select numerical columns
numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train[my_cols].copy()
X_valid = X_valid[my_cols].copy()
X_test = X_test[my_cols].copy()

print(X_train.shape), print(y_train.shape)
print(X_valid.shape), print(y_valid.shape)
print(X_test.shape), print(y_test.shape)
print(categorical_cols)
print(numerical_cols)

(250, 11)
(250,)
(31, 11)
(31,)
(32, 11)
(32,)
['COUNTY', 'STATE']
['High school graduate', 'Associates degree', 'Bachelors degree', 'High school graduate or higher', 'Bachelors degree or higher', '$25,000 to $34,999', '$35,000 to $49,999', '$50,000 to $74,999', '$75,000 to $99,999']


In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [5]:
# XGBoot
from xgboost import XGBRegressor

#model = XGBRegressor(tree_method='gpu_hist',enable_categorical=True,n_estimators=1000, learning_rate=0.05, n_jobs=4)



In [6]:
# RandonForestRegressor or DecisionTreeRegressor 
#from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(criterion="absolute_error", n_estimators=100, random_state=0)



In [7]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)

print(preds[-1])

print('MAE Baseline:', score)


18.145999999999987
MAE Baseline: 1.785612903225808


In [11]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')



print("MAE Baseline scores:\n", scores, "\n MAE Baseline Average:",scores.mean())

MAE Baseline scores:
 [1.88042857 2.43369841 2.32316667 3.46114516 2.09045161] 
 MAE Baseline Average: 2.437778084997439
